## FEM Theory and Derivations for 1D

### Grid and Indexing

We begin with the structural definition of our system.

Let the grid points start with index $i = 0$ and let the positions be $x_i$ where we set $x_0 = 0$ and the last point $x_{N_{\mathrm{pt}}-1} = L_x$.

In the non-periodic case we get $N_{\mathrm{ele}} = N_{\mathrm{pt}} - 1$ (see figure below).

Note also that the index of the basis (tent) functions corresponds to the index of the point, from which they are spanned.

$$
\varphi_j(x_j) = 1; \quad \varphi_j(x_k) = 0 \quad \forall k \neq j
$$

<img src="./figures/FEM_ST_NP.png" alt="Beschreibung" width="300">

In the periodic case, we agree on the following scheme: The domain is extended to the right, creating an additional element that 'sees' $x_0$ on its right side.

In the periodic case we get $N_{\mathrm{ele}} = N_{\mathrm{pt}}$.

<img src="./figures/FEM_ST_P.png" alt="Beschreibung" width="400">

### Basis Approach

We approximate unknowns by

$$
f_N(x) = \sum_{n=0}^{N_{\mathrm{pt}}} a_n \varphi_n(x)
$$

where $f_N(x)$ is the approximation function that includes all tent functions and their respective nodal values $a_n$. Determining these $a_n$ is our central task. Since the tent functions are linear, we effectively get a linear interpolation of nodal values for our solution. Assume that $f(x)$ is our real solution. The residual would be:

$$
R(x) = f_N(x) - f(x)
$$

<hr>

In the Galerkin approach, we demand that the residual is orthogonal to all test functions $\varphi_n$ (these are tent functions with nodal value $\varphi_n(x_n) = 1$).

$$
\int_{x_{n-1}}^{x_{n+1}} \varphi_n(x) \, R(x) \mathrm{d}x = (\varphi_n, R) = 0 \quad \forall n \in 0, ..., N_{\mathrm{pt}}
$$

For linear residual terms, closed-form expressions could be found for the integral but for non-linear terms we need to use Gaussian quadrature for approximation.

Note that we can use Gaussian quadrature also for linear terms obtaining the exact result, which is why this solver consistently uses Gaussian quadrature for all terms.

$$
\int_{-1}^{1} f(x)\,\mathrm{d}x \approx \sum_{i=1}^{n_{\mathrm{quad}}} \omega_i\,f(x_{\mathrm{quad},i})
$$

with the numer of quadrature points $n_{\mathrm{quad}}$, the corresponding weights $w_i$ and quadrature points $x_{\mathrm{quad},i}$.

<hr>

For non-linear terms, the matrix $M$ is a function of the current guess of the unknown nodal values $a_i$. Therefore we iteratively solve this scheme:

$$
\underline{M} \, \Delta \vec{a} = \vec{R}
$$

$$
\vec{a}_{i+1} = \vec{a}_i - \Delta \vec{a}
$$

where

$$
M_{ik} = \frac{\partial (\varphi_i, R)}{\partial a_k} = \frac{\partial R_i}{\partial a_k}
$$

The *tangential* matrix tells us how the integral of the individual test functions (index $i$) with residual $R$ changes by varying the nodal value of a unknown (index $j$). Note here that $\vec{a}$ comprises several variables (density, mass flux, and energy).

<hr>

Note: For careful analysis, we look at the individual terms of mass conservation, momentum and energy equation. In the following, these are referred to as `residual terms`.

When the residual term is a zeroth-order (non-linear) function of unknown variable(s), the scheme is relatively straightforward.

$$
R = f(\rho) \quad \mathrm{Case 1}
$$

However, it get's a little bit more complicated when we consider first- and second-order spatial derivatives (which we both need):

$$
R = f \left( \frac{\partial \rho}{\partial x} \right) \quad \mathrm{Case 2}
$$

$$
R = f \left( \frac{\partial^2 \rho}{\partial x^2} \right) \quad \mathrm{Case 3}
$$

In the following sections, expressions for residual vector values $R_i$ and tangential matrix entries $M_{ij}$ will be derived for the three cases above.

### Residual as zeroth-order function of unknowns

Remember $f_N$ denotes the approximated unknown function.

$$
\frac{\partial R}{\partial a_k}
= \frac{\partial R}{\partial f_N}
\frac{\partial f_N}{\partial a_k}
$$

Derivative of residual w.r.t. approximation function value is straightforward. Derivative of approximation function value w.r.t. nodal value:

$$
f_N = N_1(\xi)\, a_{\text{left}} + N_2(\xi)\, a_{\text{right}}
$$

<img src="./figures/FEM_ELE.png" alt="Beschreibung" width="300">

$$
\frac{\partial f_N}{\partial a_{\text{left}}} = N_1(\xi),
\qquad
\frac{\partial f_N}{\partial a_{\text{right}}} = N_2(\xi)
$$

with $\xi$ denoting the relative position within an element:

$$
\xi = \frac{x - x_{\mathrm{left}}}{x_{\mathrm{right}} - x_{\mathrm{left}}}
$$

Assume we iterate through test functions $\varphi_i$. The test function is spanned around $x_i$ and effectively interferese with unknown nodal values at $x_{i-1}, x_i, x_{i+1}$.
Let's look at the left and right element separately. First the element on the right side of $x_i$. Since it is the right side, the test function is represented by $N_1$ here.

$$
M_{ii,\text{right}}
= \left( N_1,\;
\frac{\partial R}{\partial f_N}
\frac{\partial f_N}{\partial a_i}
\right)
$$

In this case, $a_i = a_{\text{left}}$ and therefore:

$$
M_{ii,\text{right}}
 = \left( N_1,\;
          \frac{\partial R}{\partial f_N}\,
          N_1(\xi)
   \right)
$$

and for the left side where $a_i = a_{\text{right}}$:

$$
M_{ii,\text{left}}
 = \left( N_2,\;
          \frac{\partial R}{\partial f_N}\,
          N_2(\xi)
   \right)
$$

For the off-diagonal elements we have on the left side $a_{i-1} = a_{\text{left}}$:

$$
M_{i,i-1, \text{left}}
 = \left( N_2,\;
          \frac{\partial R}{\partial f_N}\,
          N_1(\xi)
   \right)
$$

and on the right side:

$$
M_{i,i+1,\text{right}}
 = \left( N_1,\;
          \frac{\partial R}{\partial f_N}\,
          N_2(\xi)
   \right)
$$

Note that the indices *left* and *right* are for understanding what happens in the left-side / right-side element. The overall value of $M_{ii}$ is the sum of $M_{ii,\text{left}}$ and $M_{ii,\text{right}}$. The implementation follows this scheme and iterates for each test function through the two adjacent elements.

### Residual as first spatial derivative of unknowns

For illustration, let's consider the pressure derivative term (momentum equation). The pressure itself is a function of the unknown density, which we want to approximate by $\rho_N$. In the momentum equation, we need to consider the first spatial derivative of the pressure.

$$
\frac{\partial j}{\partial t} = - \frac{\partial p}{\partial x} + ...
$$

<hr>

The residual formulation is straightforward.

$$
R = \frac{\partial p}{\partial x}= \frac{\partial p}{\partial \rho_N}
    \frac{\partial \rho_N}{\partial x}
$$

with

$$
\frac{\partial \rho_N}{\partial x} = \frac{\rho_{\text{right}} - \rho_{\text{left}}}{\Delta x}
$$

For the left side of the test function:

$$
R_{i,\text{left}}
 = \left( N_2,\; \frac{\partial p}{\partial \rho_N} \right)
   \frac{\rho_i - \rho_{i-1}}{\Delta x_{\text{left}}}
$$

And right side in the same manner. Note that $\frac{\partial p}{\partial \rho}$ is known using auto-differentiation on the equation of state.

<hr>

And what about the tangential matrix?  We need to know:

$$
\frac{\partial}{\partial a_k}
\left( \frac{\partial p}{\partial x} \right) =
\frac{\partial}{\partial a_k}
\left(
\frac{\partial p}{\partial \rho_N}
\frac{\partial \rho_N}{\partial x}
\right) =
\frac{\partial}{\partial a_k}
\left( \frac{\partial \rho_N}{\partial x} \right)
\frac{\partial p}{\partial \rho_N}
$$


And what about the tangential matrix?  We need to know:

$$
\frac{\partial}{\partial a_k}
\left( \frac{\partial p}{\partial x} \right) =
\frac{\partial}{\partial a_k}
\left(
\frac{\partial p}{\partial \rho_N}
\frac{\partial \rho_N}{\partial x}
\right) =
\frac{\partial}{\partial a_k}
\left( \frac{\partial \rho_N}{\partial x} \right)
\frac{\partial p}{\partial \rho_N}
$$

with

$$
\rho_N = N_1(\xi)\,a_{\text{left}}
       + N_2(\xi)\,a_{\text{right}}
$$

$$
\frac{\partial N_1}{\partial x} = -\frac{1}{\Delta x},
\qquad
\frac{\partial N_2}{\partial x} = \frac{1}{\Delta x}
$$

$$
\frac{\partial \rho_N}{\partial x}
 = \frac{a_{\text{right}} - a_{\text{left}}}{\Delta x}
$$

$$
\frac{\partial}{\partial a_k}
\left(
\frac{\partial \rho_N}{\partial x}
\right) =
\frac{\partial}{\partial a_k}
\left(
\frac{a_{\text{right}} - a_{\text{left}}}{\Delta x}
\right)
$$

$$
\frac{\partial}{\partial a_{\text{left}}}
\left(
\frac{\partial \rho_N}{\partial x}
\right) = -\frac{1}{\Delta x},
\qquad
\frac{\partial}{\partial a_{\text{right}}}
\left(
\frac{\partial \rho_N}{\partial x}
\right) =  \frac{1}{\Delta x}
$$

<hr>

So now we have the pieces. We want:

$$
M_{ik} = \frac{\partial\left(\varphi_i, R\right)}{\partial a_k} = \left( \varphi_i,\; \frac{\partial R}{\partial a_k} \right)
$$

For the left side element we have:

$$
\left(
N_2,\;
\frac{\partial\left(\varphi_i, R\right)}{\partial a_k}
\right) =
\left(
N_2,\;
\frac{\partial p}{\partial \rho_N}
\frac{\partial}{\partial a_k}
\left(
\frac{\partial \rho_N}{\partial x}
\right)
\right)
$$

For the diagonal entry, $a_i = a_{\text{right}}$ and therefore

$$
M_{ii,\text{left}} =
\left(
N_2,\;
\frac{\partial p}{\partial \rho_N}
\frac{1}{\Delta x_{\text{left}}}
\right)
$$

and for the right side element with $a_i = a_{\text{left}}$:

$$
M_{ii,\text{right}} =
\left(
N_1,\;
\frac{\partial p}{\partial \rho_N}
\frac{-1}{\Delta x_{\text{right}}}
\right)
$$

Off-diagonal for the left side element:

$$
M_{i, i-1,\text{left}} =
\left(
N_2,\;
\frac{\partial p}{\partial \rho_N}
\frac{-1}{\Delta x_{\text{left}}}
\right)
$$

and for the right side element:

$$
M_{i, i+1,\text{right}} =
\left(
N_1,\;
\frac{\partial p}{\partial \rho_N}
\frac{1}{\Delta x_{\text{right}}}
\right)
$$

### Residual as second-order spatial derivative of unknowns

Illustrated using the temperature diffusion term (the only term that requires this):

$$
R = \frac{\partial^2 T}{\partial x^2} = \frac{\partial}{\partial x} \frac{\partial T}{\partial x}
$$

$$
R_i = (\varphi_i, R)
$$

and with partial integration:

$$
R_i = - (\varphi'_i, \int R \, dx)
$$

$$
R_i = - \left( \frac{\partial \varphi_i}{\partial x}, \frac{\partial T}{\partial x} \right)
$$

Note that $\frac{\partial \varphi_i}{\partial x} = -1$ for $N_1$ and $\frac{\partial \varphi_i}{\partial x} = 1$ for $N_2$.

The calculation of the residual should be straightforward using

$$
\frac{\partial T}{\partial x} 
= \frac{\partial T}{\partial \rho} \frac{\partial \rho}{\partial x}
+ \frac{\partial T}{\partial j_x} \frac{\partial j_x}{\partial x}
+ \frac{\partial T}{\partial (\rho E)} \frac{\partial (\rho E)}{\partial x}
$$

Note that $\frac{\partial T}{\partial \rho} = f(\rho, j_x, \rho E)$ and is therefore non-constant over the elements.  
Keep doing quadrature with e.g. 3 points.

$$
\frac{\partial T}{\partial x} 
= \frac{1}{\Delta x} \left(
\frac{\partial T}{\partial \rho} \Delta \rho
+ \frac{\partial T}{\partial j_x} \Delta j_x
+ \frac{\partial T}{\partial (\rho E)} \Delta (\rho E)
\right)
$$

Assume we iterate over test functions, then we have for the left side element:

$$
R_{i, \text{left}} = - \left( +1, \frac{\partial T}{\partial x} \right)
$$

and for the right side:

$$
R_{i, \text{right}} = - \left( -1, \frac{\partial T}{\partial x} \right)
$$


<hr>

What about the tangential matrix?

$$
\frac{\partial R_i}{\partial a_i} 
= - \left( \frac{\partial \varphi_i}{\partial x}, \frac{\partial}{\partial a_i} \left( \frac{\partial T}{\partial x} \right) \right)
$$

This problem is similar to the first spatial derivative tangential matrix. For the left element we have:

$$
M_{ii, \text{left}, \rho} = - \left( +1, \frac{+1}{\Delta x_{\text{left}}} \left( \frac{\partial T}{\partial \rho} \right) \right)
$$

$$
M_{ii, \text{right}, \rho} 
= - \left( -1, \frac{-1}{\Delta x_{\text{left}}} \left( \frac{\partial T}{\partial \rho} \right) \right)
$$

and for the off-diagonal elements:

$$
M_{i, i-1, \text{left}, \rho} = - \left( +1, \frac{-1}{\Delta x_{\text{left}}} \left( \frac{\partial T}{\partial \rho} \right) \right)
$$

$$
M_{i, i+1, \text{right}, \rho} = - \left( -1, \frac{+1}{\Delta x_{\text{left}}} \left( \frac{\partial T}{\partial \rho} \right) \right)
$$

Note that this is only for $\rho$ and has to be repeated for the partial derivative contributions of $j_x$ and $E$.

Implementation: The residual term is treated similar to the first spatial derivative. Simply test function values are changed from zeroth-order value to first derivative value.

### Dynamic Solution

Currently solving for steady state:

$$
\mathbf{K}(\vec{a}) \cdot \Delta \vec{a} = - \vec{R}(\vec{a})
$$

$$
\mathbf{K}(\vec{a}) = \frac{\partial \vec{R}}{\partial \vec{a}}
$$

Using backwards (*implicit Euler*) time stepping, we get:

$$
\mathbf{K}(\vec{a}) \cdot \Delta \vec{a} = - \vec{R}(\vec{a}, \vec{a}_{n-1})
$$

where $\vec{a}_{n-1}$ is the solution of the previous time step. It is only required in the time derivative term:

$$
\vec{R} = \frac{\mathbf{M}}{\Delta t} (\vec{a} - \vec{a}_{n-1}) + f(\vec{a}) \dots
$$

and consequently, the tangential matrix needs to be updated:

$$
\mathbf{K}_{\text{eff}} = \mathbf{K} + \frac{\mathbf{M}}{\Delta t}
$$

## Equations 1D

Overview of 1D equations solved by the FEM framework.

Look-up guide for activating or deactivating specific terms (or "physical effects" if you want).

### Mass Conservation

Mass conservation connects density $\rho$ and mass flux $j_x$:

$$
\begin{aligned}
0 &= 
-\,\frac{\partial \left(\overline{\rho} - \overline{\rho}_{\mathrm{prev}}\right)}{\partial t} \\[4pt]
&\quad
-\,\frac{\partial \overline{j_x}}{\partial x}
- \frac{1}{h}\,\frac{\partial h}{\partial x}\,\overline{j_x} \\[4pt]
&\quad
-\,\overline{\rho}\,\frac{1}{h}\,\frac{\partial h}{\partial t}
\end{aligned}
$$

- **R1T**: Time step
- **R11**: Flux divergence + height source term
- **R12**: Height change over time

### Momentum

Momentum equation, note that $\frac{\overline{j_x}^2}{\overline{\rho}}$ represents the momentum flux.

$$
\begin{aligned}
0 &= 
- \frac{\partial (\overline{j_x})}{\partial t} \\[2pt]
&\quad
- \frac{\partial p}{\partial x} \\[2pt]
&\quad
- \frac{\partial \left( \frac{\overline{j_x}^2}{\overline{\rho}} \right) }{\partial x}
- \frac{1}{h} \frac{\partial h}{\partial x}\, \frac{\overline{j_x}^2}{\overline{\rho}} \\[2pt]
&\quad
+ \frac{\partial \tau_{xx}}{\partial x}
+ \frac{1}{h} \frac{\partial h}{\partial x}\, \left( \overline{\tau_{xx}} - \tau_{xx}\vert_{z=h_2} \right) \\[2pt]
&\quad
+ \frac{1}{h}\, \tau_{xz}\Big\vert^{z=h_2}_{z=h_1} \\[2pt]
&\quad
- \frac{1}{h} \frac{\partial h}{\partial t}\, \overline{j_x}
\end{aligned}
$$



- **R2T**: Time step
- **R21**: Pressure gradient ($p$ is constant across film, therefore no height source term)
- **R22**: Momentum flux + height source term
- **R23**: Shear in $x$ + height source term
- **R24**: Wall shear stresses
- **R25**: Height change over time

### Energy

The whole equation looks like this. Note that temperature $T$ is a function of all three conserved quantities $\rho, j_x, E$.

$$
\begin{aligned}
0 &= 
-\,\frac{\partial \left(\overline{E} - \overline{E}_{\text{prev}}\right)}{\partial t} \\[6pt]
&\quad
-\,\frac{\partial (\overline{E u})}{\partial x}
- \overline{E u}\,\frac{1}{h}\,\frac{\partial h}{\partial x} \\[6pt]
&\quad
-\,\frac{\partial (p\,\overline{u})}{\partial x}
- p\,\overline{u}\,\frac{1}{h}\,\frac{\partial h}{\partial x} \\[6pt]
&\quad
+\,\frac{\partial}{\partial x}\!\left(\overline{\tau_{xx}}\,\overline{u}\right)
+ \left(\overline{\tau_{xx}}\,\overline{u}\right)\frac{1}{h}\,
\frac{\partial h}{\partial x} \\[6pt]
&\quad
-\,\frac{1}{h}\left(\tau_{xz,\text{bot}}\,U_{\text{bot}}\right) \\[6pt]
&\quad
+\,k\,\frac{\partial^2 \overline{T}}{\partial x^2} \\[6pt]
&\quad
+ S_{\text{walls}} \\[6pt]
&\quad
- \overline{E}\,\frac{1}{h}\,\frac{\partial h}{\partial t}
\end{aligned}
$$

- **R3T**: Time step
- **R31**: Energy advection + height source term
- **R32**: Pressure Work + height source term
- **R33**: Shear work $xx$ + height source term
- **R34**: Wall shear work $xz$: work done by wall movement
- **R35**: Thermal diffusion
- **R36**: Wall heat balance
- **R37**: Height change over time